In [39]:
import requests
import fitz  # PyMuPDF
from io import BytesIO
import os
import re
import logging
from typing import List, Dict
from dotenv import load_dotenv

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker, LLMChainExtractor, EmbeddingsFilter, DocumentCompressorPipeline 
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain_community.vectorstores.utils import filter_complex_metadata

In [40]:
file=os.path.join(os.path.dirname(os.getcwd()), 'servelec/70993873-07ae-4e2b-b1f9-b08858ff1ddb.pdf')
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

In [42]:
def read_pdf_from_file(file_path) -> List[Dict]:
    """Read PDF from file path and extract text with page metadata"""
    try:
        pdf_document = fitz.open(file_path)
        
        pages_data = []
        for page_num in range(pdf_document.page_count):
            page = pdf_document[page_num]
            page_text = page.get_text()
            pages_data.append({
                'text': page_text,
                'page_number': page_num + 1,
                'total_pages': pdf_document.page_count
            })
        
        pdf_document.close()
        return pages_data
        
    except Exception as e:
        print(f"Error reading PDF from file: {e}")
        return None

In [41]:
def chunk_text(self, pages_data: List[Dict], chunk_size: int = 1000, chunk_overlap: int = 200) -> List[Dict]:
        """Split text into chunks with semantic metadata"""
        if not pages_data:
            return []
        
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            length_function=len,
            separators=["\n\n", "\n", " ", ""]
        )
        
        chunks_with_metadata = []
        for page_data in pages_data:
            if not page_data['text'].strip():
                continue
                
            chunks = text_splitter.split_text(page_data['text'])
            for i, chunk in enumerate(chunks):
                
                # Combine with page metadata
                chunk_metadata = {
                    'page_number': page_data['page_number'],
                    'total_pages': page_data['total_pages'],
                    'chunk_index': i
                }
                doc={}
                # Create Document object
                doc['page_content']=chunk
                doc['metadata']=chunk_metadata
                chunks_with_metadata.append(doc)
        
        return chunks_with_metadata

In [43]:
pages_data = read_pdf_from_file(file)
chunks = chunks_with_metadata = chunk_text(None, pages_data)

In [44]:
len(chunks)

112

In [49]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
prompt = ChatPromptTemplate.from_template("""
You are a technical content classifier for electrical and electronics
  tender documents. Your task is to categorize each text chunk as either
   "TECHNICAL" or "NON-TECHNICAL".

  **TECHNICAL** chunks contain:
  - Product specifications, technical requirements, or performance
  criteria
  - Equipment descriptions, models, or part numbers
  - Electrical parameters (voltage, current, power, frequency, etc.)
  - Technical standards, certifications, or compliance requirements
  - Installation, configuration, or operational procedures
  - System architecture, wiring diagrams, or technical drawings
  - Quality assurance, testing, or measurement specifications
  - Materials, components, or manufacturing details
  - Technical service requirements or maintenance procedures

  **NON-TECHNICAL** chunks contain:
  - Administrative procedures, legal terms, or contractual conditions
  - Payment terms, delivery schedules, or commercial conditions
  - General company information or background
  - Bidding instructions or submission requirements
  - Insurance, liability, or legal compliance (non-technical)
  - Project timelines or administrative milestones
  - General introductory or concluding statements
  - Contact information or communication protocols

  **Instructions:**
  1. Read the text chunk carefully
  2. Respond with only "TECHNICAL" or "NON-TECHNICAL"
  3. When in doubt, err on the side of "TECHNICAL" to ensure only
  clearly technical content passes through

  **Text chunk to classify:**
  {text_chunk}
  **Classification:**
"""
)

In [50]:
len(chunks)

112

In [51]:
chain = prompt | llm | StrOutputParser()
#example_chunk = chunks[0]
#classification = chain.invoke({"text_chunk": example_chunk['page_content']})
#example_chunk['classification']=classification

In [35]:
example_chunk

{'page_content': "Procedimiento General  \nLICITACION PUBLICA ELECTRONICA  \nCódigo: DP-LPE-01  \nRev 0 \nFecha de Vigencia: \n06/06/2023  \n \n \nEmitió:   \nReferente de \nCalidad  \nRevisó:   \nResponsable del Proceso  \nAprobó:   \nAutoridad de EPEC \n \n \n \nEMPRESA PROVINCIAL DE ENERGÍA DE CÓRDOBA \nLICITACION PÚBLICA ELECTRONICA Nº  \nSIC N° 20084 \n \n \nOBJETO: “ADQUISICIÓN CARGADORES Y \nRECTIFICADORES” \n \n \nDigitally signed by TORRE Lucia\nDN: SERIALNUMBER=CUIL \n27338273210, C=AR, CN=\nTORRE Lucia\nReason: I have reviewed this \ndocument\nLocation: EPEC, Cordoba\nDate: 2024.07.05 \n14:44:30\n-03'00'\nFoxit PDF Reader Version: \n2024.1.0\nTORR\nE Lucia",
 'metadata': {'page_number': 1, 'total_pages': 36, 'chunk_index': 0},
 'classification': 'NON-TECHNICAL'}

In [36]:
print(example_chunk['page_content'])

Procedimiento General  
LICITACION PUBLICA ELECTRONICA  
Código: DP-LPE-01  
Rev 0 
Fecha de Vigencia: 
06/06/2023  
 
 
Emitió:   
Referente de 
Calidad  
Revisó:   
Responsable del Proceso  
Aprobó:   
Autoridad de EPEC 
 
 
 
EMPRESA PROVINCIAL DE ENERGÍA DE CÓRDOBA 
LICITACION PÚBLICA ELECTRONICA Nº  
SIC N° 20084 
 
 
OBJETO: “ADQUISICIÓN CARGADORES Y 
RECTIFICADORES” 
 
 
Digitally signed by TORRE Lucia
DN: SERIALNUMBER=CUIL 
27338273210, C=AR, CN=
TORRE Lucia
Reason: I have reviewed this 
document
Location: EPEC, Cordoba
Date: 2024.07.05 
14:44:30
-03'00'
Foxit PDF Reader Version: 
2024.1.0
TORR
E Lucia


In [55]:
for chunk in chunks:
    classification = chain.invoke({"text_chunk": chunk['page_content']})
    chunk['classification']=classification

In [58]:
from __future__ import annotations

from decimal import Decimal
from enum import Enum
from typing import Optional

from pydantic import BaseModel, Field, ConfigDict, field_validator


# ---------- Reusable types ----------

class NumericRange(BaseModel):
    """Generic numeric range with optional unit label."""
    model_config = ConfigDict(extra='forbid')

    min: Decimal = Field(..., description="Valor mínimo")
    max: Decimal = Field(..., description="Valor máximo")
    unidad: Optional[str] = Field(None, description="Unidad (V, A, Hz, °C, kVA, etc.)")

    @field_validator('max')
    @classmethod
    def _min_le_max(cls, v, info):
        min_val = info.data.get('min')
        if min_val is not None and v < min_val:
            raise ValueError('max debe ser >= min')
        return v


class Dimensiones(BaseModel):
    model_config = ConfigDict(extra='forbid')

    ancho_mm: Optional[Decimal] = Field(None, description="Ancho en mm")
    alto_mm: Optional[Decimal] = Field(None, description="Alto en mm")
    profundidad_mm: Optional[Decimal] = Field(None, description="Profundidad en mm")


class InterruptorSpec(BaseModel):
    model_config = ConfigDict(extra='forbid')

    tipo: Optional[str] = Field(None, description="Tipo de interruptor")
    calibre_a: Optional[Decimal] = Field(None, description="Calibre en amperes (A)")


class FaseTipo(str, Enum):
    monofasica = "monofásica"
    bifasica = "bifásica"
    trifasica = "trifásica"


class ConexionNeutroTierra(str, Enum):
    TT = "TT"
    TN = "TN"
    IT = "IT"
    otro = "otro"


class ServicioTipo(str, Enum):
    continuo = "continuo"
    intermitente = "intermitente"
    otro = "otro"


class AlimentacionAlternativaTipo(str, Enum):
    interna = "interna"
    externa = "externa"


# ---------- 1 Generales ----------

class Generales(BaseModel):
    model_config = ConfigDict(extra='forbid')

    codigo_producto: Optional[str] = None
    marca: Optional[str] = None
    modelo: Optional[str] = None
    normas_fabricacion: Optional[str] = Field(None, description="Normas / estándares aplicables")
    origen: Optional[str] = Field(None, description="País o región de fabricación")
    tipo: Optional[str] = Field(None, description="Familia / categoría del equipo")
    apto_baterias_pb_y_nicd: Optional[bool] = Field(None, description="Apto para Pb-Ca/Pb-Ac y Ni-Cd")


# ---------- 2 Condiciones ambientales de servicio ----------

class CondicionesAmbientales(BaseModel):
    model_config = ConfigDict(extra='forbid')

    temperatura_max_c: Optional[Decimal] = Field(None, description="Temperatura máxima de operación (°C)")
    temperatura_min_c: Optional[Decimal] = Field(None, description="Temperatura mínima de operación (°C)")
    altura_snm_m: Optional[Decimal] = Field(None, description="Altura sobre el nivel del mar (m)")
    humedad_relativa_max_pct: Optional[Decimal] = Field(None, description="Humedad relativa máx. sin condensación (%)")
    instalacion: Optional[str] = Field(None, description="Interior/Exterior/Armario/etc.")
    tipo_servicio: Optional[ServicioTipo] = None


# ---------- 3 Alimentación (entrada) ----------

class Alimentacion(BaseModel):
    model_config = ConfigDict(extra='forbid')

    tipo: Optional[FaseTipo] = Field(None, description="Tipo de fase")
    tension_v: Optional[Decimal] = Field(None, description="Tensión nominal (V)")
    rango_tension_entrada: Optional[NumericRange] = Field(None, description="Rango de tensión admisible (V)")
    frecuencia_hz: Optional[Decimal] = Field(None, description="Frecuencia nominal (Hz)")
    rango_frecuencia_entrada: Optional[NumericRange] = Field(None, description="Rango de frecuencia admisible (Hz)")

    conexion_neutro_tierra: Optional[ConexionNeutroTierra] = Field(None, description="Esquema de conexión a tierra")
    conductor_pe_independiente: Optional[bool] = Field(None, description="PE independiente del neutro")
    isc_punto_conexion: Optional[Decimal] = Field(None, description="Corriente de cortocircuito en punto de conexión (A o kA)")
    interruptor_acometida: Optional[InterruptorSpec] = Field(None, description="Interruptor de acometida")
    potencia_transformador_entrada_kva: Optional[Decimal] = Field(None, description="Potencia del transformador de entrada (kVA)")
    corriente_conexion_transformador_a: Optional[Decimal] = Field(None, description="Corriente de conexión del transformador (A)")


# ---------- 4 Salida (DC bus/consumos) ----------

class Salida(BaseModel):
    model_config = ConfigDict(extra='forbid')

    tension_nominal_v: Optional[Decimal] = Field(None, description="Tensión nominal de salida (V)")
    corriente_nominal_a: Optional[Decimal] = Field(None, description="Corriente nominal de salida (A)")
    maxima_corriente_consumos_a: Optional[Decimal] = Field(None, description="Máxima corriente a consumos (A)")


# ---------- 5 Carga de baterías / tensiones ----------

class ModosCarga(BaseModel):
    model_config = ConfigDict(extra='forbid')

    manual_automatico: Optional[str] = Field(None, description="Modo de operación: manual/automático")
    carga_excepcional: Optional[bool] = Field(None, description="Disponibilidad de carga excepcional")


class RangoTensionPorQuimica(BaseModel):
    model_config = ConfigDict(extra='forbid')

    nicd: Optional[NumericRange] = Field(None, description="Rango de tensión para Ni-Cd (V)")
    pb_ca: Optional[NumericRange] = Field(None, description="Rango de tensión para Pb-Ca (V)")


class CargaBaterias(BaseModel):
    model_config = ConfigDict(extra='forbid')

    tension_flote: Optional[NumericRange] = Field(None, description="Tensión de flote (rango de ajuste, V)")
    estabilizacion_tension_flote: Optional[str] = Field(None, description="Estabilización de tensión de flote (ej., ±1%)")
    tension_fondo: Optional[NumericRange] = Field(None, description="Tensión de fondo (rango de ajuste, V)")
    modos_carga: Optional[ModosCarga] = None
    regulador_diodos: Optional[bool] = Field(None, description="Regulación mediante cadena de diodos")
    rango_tension_salida_consumo: Optional[RangoTensionPorQuimica] = Field(None, description="Rango de tensión de salida a consumo por química")
    deteccion_polo_tierra: Optional[bool] = Field(None, description="Detección de polo a tierra")
    interruptor_salida_consumo: Optional[InterruptorSpec] = Field(None, description="Interruptor de salida a consumo")
    interruptor_salida_baterias: Optional[InterruptorSpec] = Field(None, description="Interruptor de salida a baterías")
    sistema_rectificacion: Optional[str] = Field(None, description="Tecnología del rectificador")
    ripple_con_baterias: Optional[str] = Field(None, description="Nivel de rizado con baterías (mVpp o %)")
    ripple_sin_baterias: Optional[str] = Field(None, description="Nivel de rizado sin baterías (mVpp o %)")


# ---------- 6 Gabinete del sistema ----------

class Gabinete(BaseModel):
    model_config = ConfigDict(extra='forbid')

    material: Optional[str] = None
    acceso: Optional[str] = Field(None, description="Acceso: frontal/lateral/trasero")
    grado_proteccion: Optional[str] = Field(None, description="IP/NEMA")
    espesor_chapa_estructura_mm: Optional[Decimal] = Field(None, description="Espesor de chapa (mm)")
    tipo_pintura: Optional[str] = None
    color: Optional[str] = Field(None, description="Código RAL u otro")
    espesor_pintura_micras: Optional[Decimal] = Field(None, description="Espesor de pintura (μm)")
    dimensiones: Optional[Dimensiones] = Field(None, description="Dimensiones (mm)")


# ---------- 7 Otros ----------

class Otros(BaseModel):
    model_config = ConfigDict(extra='forbid')

    ventilacion: Optional[str] = Field(None, description="Natural/forzada")
    rectificador: Optional[str] = Field(None, description="Topología o modelo del rectificador")


# ---------- 8 Protecciones ----------

class Protecciones(BaseModel):
    model_config = ConfigDict(extra='forbid')

    sobretensiones_cc: Optional[bool] = Field(None, description="Protección contra sobretensiones en CC")
    sobretensiones_ca: Optional[bool] = Field(None, description="Protección contra sobretensiones en CA")
    cortocircuito: Optional[bool] = Field(None, description="Protección contra cortocircuito")
    sobrecarga: Optional[bool] = Field(None, description="Protección contra sobrecarga")
    lvd: Optional[bool] = Field(None, description="Desconexión por baja tensión (LVD)")


# ---------- 9 Alarmas ----------

class Alarmas(BaseModel):
    model_config = ConfigDict(extra='forbid')

    falla_red: Optional[bool] = None
    alta_tension_baterias: Optional[bool] = None
    baja_tension_baterias: Optional[bool] = None
    tension_baterias_critica: Optional[bool] = Field(None, description="Batería descargada")
    alta_tension_consumos: Optional[bool] = None
    baja_tension_consumos: Optional[bool] = None
    tension_consumo_critica: Optional[bool] = None
    falla_rectificador: Optional[bool] = None
    alta_corriente_rectificador: Optional[bool] = None
    alta_corriente_baterias: Optional[bool] = None
    alta_corriente_consumos: Optional[bool] = None
    polo_positivo_tierra: Optional[bool] = None
    polo_negativo_tierra: Optional[bool] = None
    fallo_ventilador: Optional[bool] = Field(None, description="Si corresponde")
    obstruccion_filtros: Optional[bool] = Field(None, description="Si corresponde")
    bateria_en_descarga: Optional[bool] = None
    averia_red_salida: Optional[bool] = Field(None, description="Falla de aislación")
    apertura_interruptores: Optional[bool] = None
    fallo_bateria_supervision: Optional[bool] = Field(None, description="Si hay sistema de supervisión")
    temperatura_termostato: Optional[bool] = None
    cargador_modo_fondo: Optional[bool] = None


# ---------- 10 Señalizaciones ----------

class Senalizaciones(BaseModel):
    model_config = ConfigDict(extra='forbid')

    red_ok: Optional[bool] = Field(None, description="Presencia de tensión alterna en redes")
    bateria_carga_flotacion: Optional[bool] = None
    bateria_carga_rapida: Optional[bool] = None
    bateria_carga_excepcional: Optional[bool] = None
    ventiladores_funcionando: Optional[bool] = Field(None, description="Si aplica")
    carga_por_red_alternativa: Optional[bool] = None
    falla: Optional[bool] = None
    tension_bateria: Optional[bool] = None
    tension_consumos: Optional[bool] = None
    corriente_baterias: Optional[bool] = None
    corriente_consumos: Optional[bool] = None
    alarmas_discriminadas: Optional[bool] = None


# ---------- 11 Aparatos de medida ----------

class AparatosDeMedida(BaseModel):
    model_config = ConfigDict(extra='forbid')

    unidad_digital_centralizada: Optional[bool] = None
    protocolo_comunicacion: Optional[str] = None
    puerto_comunicacion: Optional[str] = None
    mide_corrientes_entrada: Optional[bool] = None
    mide_tensiones_entrada: Optional[bool] = None
    mide_corriente_salida_rectificador: Optional[bool] = None
    mide_corriente_carga_baterias: Optional[bool] = None
    mide_tension_salida_rectificador: Optional[bool] = None
    mide_tension_baterias: Optional[bool] = None
    mide_tension_consumos: Optional[bool] = None
    mide_corriente_descarga_baterias: Optional[bool] = None


# ---------- 12 Características, accesorios y auxiliares ----------

class ResistenciasCalefactoras(BaseModel):
    model_config = ConfigDict(extra='forbid')

    tension_alimentacion_v: Optional[Decimal] = Field(None, description="V")
    potencia_w: Optional[Decimal] = Field(None, description="W")
    llave_termomagnetica_independiente: Optional[bool] = Field(None, description="Llave dedicada")
    alimentacion: Optional[AlimentacionAlternativaTipo] = Field(None, description="Interna o externa")


class CablesPotenciaAuxiliares(BaseModel):
    model_config = ConfigDict(extra='forbid')

    tension_aislacion_v: Optional[Decimal] = Field(None, description="Tensión de aislación (V)")
    material: Optional[str] = None
    baja_emision_halogenos: Optional[bool] = None


class CaracteristicasAccesoriosAuxiliares(BaseModel):
    model_config = ConfigDict(extra='forbid')

    panel_control: Optional[str] = Field(None, description="Ajuste y configuración")
    resistencias_calefactoras: Optional[ResistenciasCalefactoras] = None
    cables_potencia_auxiliares: Optional[CablesPotenciaAuxiliares] = None
    bornes_reserva: Optional[int] = Field(None, description="Cantidad de bornes de reserva")
    nivel_ruido_db_a: Optional[Decimal] = Field(None, description="Nivel de ruido a 1 m y 100% carga (dB(A))")
    placas_identificacion: Optional[bool] = None
    chapa_caracteristicas: Optional[bool] = None


# ---------- 13 Inspecciones y ensayos de rutina ----------

class InspeccionesEnsayos(BaseModel):
    model_config = ConfigDict(extra='forbid')

    resistencia_aislacion: Optional[bool] = Field(None, description="Medición de resistencia de aislación")
    ensayo_dieletrico: Optional[bool] = Field(None, description="Ensayo dieléctrico de potencia/control (excluye electrónicos)")
    resistencia_aislacion_post: Optional[bool] = Field(None, description="Medición posterior al ensayo dieléctrico")
    elementos_mecanicos_enclavamientos: Optional[bool] = Field(None, description="Funcionamiento y enclavamientos")
    tolerancia_tension_salida: Optional[bool] = None
    nivel_rizado: Optional[bool] = None
    verificacion_valores_salida_control: Optional[bool] = Field(None, description="Verificación eléctrica en condiciones de carga")
    verificacion_tensiones_carga: Optional[bool] = Field(None, description="Verifica flote/rápida/excepcional")
    variacion_tension_alimentacion: Optional[bool] = Field(None, description="No varía en límites de alimentación")
    variacion_carga_0_a_100: Optional[bool] = Field(None, description="No varía del 0% al 100% de carga")
    verificacion_carga_bateria: Optional[bool] = None
    contenido_armonicos_alimentacion: Optional[bool] = None
    ensayo_plena_carga_48h: Optional[bool] = Field(None, description="Ensayo a plena carga hasta régimen estable")
    ensayo_rendimiento: Optional[bool] = None
    reparto_intensidades_paralelo: Optional[bool] = Field(None, description="Para cargadores en paralelo")
    prueba_senalizacion_alarma: Optional[bool] = None
    prueba_funcional_total: Optional[bool] = Field(None, description="Todas las condiciones de operación")
    capacidad_corriente_cortocircuito: Optional[bool] = Field(None, description="Capacidad para abrir el mayor interruptor de salida")


# ---------- 14 Garantía ----------

class Garantia(BaseModel):
    model_config = ConfigDict(extra='forbid')

    descripcion: Optional[str] = Field(None, description="Términos de garantía")
    meses: Optional[int] = Field(None, description="Duración en meses")


# ---------- Top-level ----------

class SistemaCargadorRectificador(BaseModel):
    """
    Esquema de hoja de datos del sistema (cargador/rectificador + baterías).
    Todos los campos son opcionales para permitir carga progresiva.
    """
    model_config = ConfigDict(extra='forbid')

    generales: Optional[Generales] = None
    condiciones_ambientales: Optional[CondicionesAmbientales] = None
    alimentacion: Optional[Alimentacion] = None
    salida: Optional[Salida] = None
    carga_baterias: Optional[CargaBaterias] = None
    gabinete: Optional[Gabinete] = None
    otros: Optional[Otros] = None
    protecciones: Optional[Protecciones] = None
    alarmas: Optional[Alarmas] = None
    senalizaciones: Optional[Senalizaciones] = None
    aparatos_medida: Optional[AparatosDeMedida] = None
    caracteristicas_accesorios: Optional[CaracteristicasAccesoriosAuxiliares] = None
    inspecciones_ensayos: Optional[InspeccionesEnsayos] = None
    garantia: Optional[Garantia] = None


# --- Ejemplo rápido ---
structured_prompt = ChatPromptTemplate.from_template("""
You are an expert data extraction system for electrical and 
  electronics tender documents. 

  Extract all relevant technical information from the provided text 
  chunk and structure it according to the SistemaCargadorRectificador 
  schema.

  **Instructions:**
  1. Extract only information that is explicitly stated in the text
  2. Use appropriate data types (Decimal for numbers, enums for 
  categorical values)
  3. Leave fields as None if information is not present or unclear
  4. For numeric ranges, extract both min and max values with units
  5. Be precise with technical specifications and measurements

  **Key categories to look for:**
  - Product specifications (marca, modelo, código)
  - Environmental conditions (temperatura, humedad, altitud)
  - Electrical parameters (tensión, corriente, potencia, frecuencia)
  - Technical standards and certifications
  - Installation and operational requirements
  - Physical dimensions and materials
  - Protection systems and alarms
  - Measurement instruments and communication protocols

  **Text chunk:**
  {text_chunk}

  Extract and structure all available technical data from this text.
"""
)
structured_chain = structured_prompt | llm.with_structured_output(SistemaCargadorRectificador)


In [59]:
technical_chunks = [chunk for chunk in chunks if chunk['classification'] == 'TECHNICAL']
for technical_chunk in technical_chunks:
    try:
        structured_data = structured_chain.invoke({"text_chunk": technical_chunk['page_content']})
        technical_chunk['structured_data'] = structured_data
    except Exception as e:
        technical_chunk['structured_data'] = None
        logging.error(f"Error processing chunk {technical_chunk['metadata']}: {e}")

In [72]:
answer=""
for chunk in technical_chunks:
    answer+=f"Document page {chunk['metadata']['page_number']} chunk {chunk['metadata']['chunk_index']}\n"
    answer+=f"Document content: {chunk['page_content']}\n"
    answer+=f"Extracted data: {chunk['structured_data']}\n"
    answer+="--------------------------------------------------\n\n"

print(answer)

Document page 2 chunk 1
Document content: g. “CONTRATISTA” O “PROVEEDOR" a la/s persona/s con las que contrate la EPEC en los 
términos del Régimen Legal aplicable a esta contratación;   
h. "INSPECCIÓN" al acto realizado por el personal de la EPEC a cuyo cargo esté el control 
y aprobación de los bienes y servicios objeto del contrato; 
i. 
“BOE” al Boletín Oficial Electrónico de la Provincia de Córdoba.  
Art. 3. Objeto de la contratación: El objeto del presente procedimiento es la “ADQUISICIÓN 
CARGADORES Y RECTIFICADORES” conforme las Especificaciones Técnicas y demás documentos 
del legajo licitatorio, en los términos del Decreto Provincial N° 399/2019.   
Descripción de lo solicitado: 
Renglón 
Cant 
Uni 
Descripción 
1 
4 
Pza 00013113 RECTIFICADOR CARGADOR DE BATERIA 
1 
2 
Eqp 00013044 EQUIPO RECTIFICADOR- CARGADOR DE BATERIAS DE 
S.A. 380V CA/ 110VCCREGULACION ELECTRONICA 
3 
1 
Eqp 00014249 EQUIPO RECTIFICADOR- CARGADOR DE BATERIAS DE 
S.A. 220V CA/ 110VCC REGULACION ELECTRO

In [73]:
with open("answer.txt", "w", encoding="utf-8") as f:
    f.write(answer)